#### Validation for lake level derived by SWOT data.
##### 1. lake levels of SWOT pixc data, SWOT LakeSp data, ICESat-2 ATL13 data and DAHITI data.
##### 2. Lake levels by using simple average and area-weighted average method.

In [27]:
import pickle  
import xarray as xr  
import pandas as pd  
from utils.date_transform import decimal_to_date


### 1. swot pixc vs. swot lakesp vs. dahiti 

In [28]:
## dianchi
path_DAHITI_dianchi= 'data/dahiti/Dan Chi.nc'  
path_pixc_wse_dianchi = 'data/swot_l2/pixc/dianchi-lake/dianchi_pixc_wse.pkl'  
path_LakeSP_wse_dianchi = 'data/swot_l2/lakesp/dianchi-lake/dianchi_wse_lakesp.pkl'
path_dahiti_pix_lakesp_dianchi = 'data/eva_data/dahiti_pixc_lakesp_dianchi.csv' ## save to path

## erhai
path_DAHITI_erhai = 'data/dahiti/ErHai.nc'
path_pixc_wse_erhai = 'data/swot_l2/pixc/erhai-lake/erhai_pixc_wse.pkl'
path_LakeSP_wse_erhai = 'data/swot_l2/lakesp/erhai-lake/erhai_wse_lakesp.pkl'
path_dahiti_pix_lakesp_erhai = 'data/eva_data/dahiti_pixc_lakesp_erhai.csv' ## save to path


#### 1.1 DAHITI水位数据处理（转换基准为EGM2008）

In [29]:
def read2month_dahiti(path_dahiti):
  '''
  (1) read dahiti data and convert to dataframe
  (2) convert date to monthly date
  (3) convert elevation reference of eigen-6c4 to egm2008
  '''
  dahiti_xr = xr.open_dataset(path_dahiti)
  dahiti_df = dahiti_xr.to_dataframe()
  dahiti_df['datetime'] = pd.to_datetime(dahiti_df['datetime'])
  dahiti_df['year_month'] = dahiti_df['datetime'].dt.to_period('M') 
  dahiti_wse_month = dahiti_df.groupby('year_month')['water_level'].mean().reset_index()
  dahiti_wse_month = dahiti_wse_month.rename(columns={'water_level': 'dahiti_wse'})
  dahiti_wse_month = dahiti_wse_month.set_index('year_month')
  #基准由DAHITI的EIGEN-6C4转为wgs84,再转为EGM2008
  dahiti_wse_month['dahiti_wse'] = dahiti_wse_month['dahiti_wse'] + (-32.2242)-(-32.1375)
  return dahiti_wse_month

dahiti_wse_month_dianchi = read2month_dahiti(path_DAHITI_dianchi)
dahiti_wse_month_erhai = read2month_dahiti(path_DAHITI_erhai)
dahiti_wse_month_erhai


,dahiti_wse
year_month,
2016-05,1965.288208
2016-06,1965.083252
2016-07,1965.069214
2016-08,1965.178711
2016-09,1965.405151
...,...
2024-08,1965.119263
2024-09,1965.403198
2024-10,1965.537231


#### 1.2 基于SWOT Pixc数据的水位信息

In [30]:
def read2month(path_lake_wse, var_name='lake_wse'):
    '''
    (1) read pixc/lakesp/icesat2 data and convert to dataframe
    (2) convert decimal date to monthly date
    '''
    with open(path_lake_wse, 'rb') as file:  
        pixc_data = pickle.load(file)      
        df = pd.DataFrame.from_dict(pixc_data, orient='index').reset_index()  
        df.columns = ['decimal_date', var_name]          
        # 转换日期
        df['date'] = df['decimal_date'].apply(decimal_to_date)  
        df['year_month'] = pd.to_datetime(df['date']).dt.to_period('M') 
        lake_wse_month = df.groupby('year_month')[var_name].mean().reset_index()    
        lake_wse_month = lake_wse_month.set_index('year_month')
    return lake_wse_month


In [31]:
pixc_wse_month_dianchi = read2month(path_pixc_wse_dianchi, var_name='swot_pixc_wse')
pixc_wse_month_erhai = read2month(path_pixc_wse_erhai, var_name='swot_pixc_wse')
pixc_wse_month_dianchi


,swot_pixc_wse
year_month,
2023-08,1887.866699
2023-09,1887.759888
2023-10,1887.797363
2023-11,1887.750366
2023-12,1887.763916
2024-01,1887.703613
2024-02,1887.785645
2024-03,1887.715210
2024-04,1887.640259


#### 1.3 基于LakeSP数据的SWOT水位信息

In [32]:
lakeSP_wse_month_dianchi = read2month(path_LakeSP_wse_dianchi, var_name='swot_lakesp_wse')
lakeSP_wse_month_erhai = read2month(path_LakeSP_wse_erhai, var_name='swot_lakesp_wse')
lakeSP_wse_month_dianchi


,swot_lakesp_wse
year_month,
2023-08,1887.907000
2023-09,1887.924500
2023-10,1887.983333
2023-11,1888.191500
2023-12,1887.847500
2024-01,1888.042000
2024-02,1887.939500
2024-03,1887.800000
2024-04,1887.781000


#### 1.4 数据合并及对比分析   
合并数据--均方根计算--可视化

In [33]:
## merge_dahiti_pix_lakesp
def merge_dahiti_pix_lakesp(dahiti_wse_month, pixc_wse_month, LakeSP_wse_month):
    '''
    (1) merge dahiti, pixc, lakesp, isat2 data into one dataframe
    (2) interpolate missing values
    (3) system bias correction for pixc and lakesp data by use dahiti data as reference
    '''
    ## 合并三个水位产品
    dahiti_pixc_lakesp_df = pixc_wse_month.merge(dahiti_wse_month, on='year_month', how='left').\
                                                merge(LakeSP_wse_month, on='year_month', how='left')
    dahiti_pixc_lakesp_df = dahiti_pixc_lakesp_df.interpolate(method='linear', limit_area='inside') 
    dahiti_pixc_lakesp_df = dahiti_pixc_lakesp_df.reindex(columns=['dahiti_wse', 'swot_pixc_wse', 'swot_lakesp_wse'])

    #计算均方根误差和相关性
    dif_dahiti_lakesp = dahiti_pixc_lakesp_df['swot_lakesp_wse'] \
                                    - dahiti_pixc_lakesp_df['dahiti_wse']
    dif_dahiti_pix = dahiti_pixc_lakesp_df['swot_pixc_wse'] \
                                    - dahiti_pixc_lakesp_df['dahiti_wse']
    ## 计算均方根误差
    ## 计算差值的均值(系统偏差，以dahiti为参考)
    mean_dif_dahiti_lakesp = dif_dahiti_lakesp.mean()
    mean_dif_dahiti_pix = dif_dahiti_pix.mean()

    # 将均值加到wse_month列（以DAHITI为参考, 消除系统偏差）
    dahiti_pixc_lakesp_df['swot_pixc_wse_cor'] = dahiti_pixc_lakesp_df['swot_pixc_wse'] - mean_dif_dahiti_pix
    dahiti_pixc_lakesp_df['swot_lakesp_wse_cor'] = dahiti_pixc_lakesp_df['swot_lakesp_wse'] - mean_dif_dahiti_lakesp
    ### write out
    return dahiti_pixc_lakesp_df

dahiti_pixc_lakesp_dianchi = merge_dahiti_pix_lakesp(dahiti_wse_month_dianchi, \
                                                           pixc_wse_month_dianchi, lakeSP_wse_month_dianchi)
dahiti_pixc_lakesp_erhai = merge_dahiti_pix_lakesp(dahiti_wse_month_erhai, \
                                                         pixc_wse_month_erhai, \
                                                         lakeSP_wse_month_erhai)
dahiti_pixc_lakesp_dianchi
# ## save to csv
# dahiti_pixc_lakesp_dianchi.to_csv(path_dahiti_pix_lakesp_dianchi, index=True)
# dahiti_pixc_lakesp_erhai.to_csv(path_dahiti_pix_lakesp_erhai, index=True)


,dahiti_wse,swot_pixc_wse,swot_lakesp_wse,swot_pixc_wse_cor,swot_lakesp_wse_cor
year_month,,,,,
2023-08,1887.499268,1887.866699,1887.907000,1887.583130,1887.463708
2023-09,1887.431152,1887.759888,1887.924500,1887.476318,1887.481208
2023-10,1887.530151,1887.797363,1887.983333,1887.513794,1887.540041
2023-11,1887.597168,1887.750366,1888.191500,1887.466797,1887.748208
2023-12,1887.448242,1887.763916,1887.847500,1887.480347,1887.404208
2024-01,1887.426147,1887.703613,1888.042000,1887.420044,1887.598708
2024-02,1887.460205,1887.785645,1887.939500,1887.502075,1887.496208
2024-03,1887.459229,1887.715210,1887.800000,1887.431641,1887.356708
2024-04,1887.319214,1887.640259,1887.781000,1887.356689,1887.337708


In [34]:
## accuracy
## dianchi
dif_pix_dahiti = dahiti_pixc_lakesp_dianchi['dahiti_wse']-dahiti_pixc_lakesp_dianchi['swot_pixc_wse_cor']
dif_lakesp_dahiti = dahiti_pixc_lakesp_dianchi['dahiti_wse']-dahiti_pixc_lakesp_dianchi['swot_lakesp_wse_cor']
print('dianchi(pixc):', dif_pix_dahiti.std(), 'dianchi(lakesp):', dif_lakesp_dahiti.std())
## erhai
dif_pix_dahiti = dahiti_pixc_lakesp_erhai['dahiti_wse']-dahiti_pixc_lakesp_erhai['swot_pixc_wse_cor']
dif_lakesp_dahiti = dahiti_pixc_lakesp_erhai['dahiti_wse']-dahiti_pixc_lakesp_erhai['swot_lakesp_wse_cor']
print('erhai(pixc):', dif_pix_dahiti.std(), 'erhai(lakesp):', dif_lakesp_dahiti.std())


dianchi(pixc): 0.058899302 dianchi(lakesp): 0.08424942434418105
erhai(pixc): 0.08635876 erhai(lakesp): 0.13366278634413


### 2. merge isat-2 wse and pix wse.

In [38]:
path_isat2_wse_dianchi = 'data/isat2/dianchi/dianchi_wse_isat2.pkl' 
path_isat2_wse_erhai = 'data/isat2/erhai/erhai_wse_isat2.pkl' 
path_pixc_isat2_dianchi = 'data/eva_data/pixc_isat2_dianchi.csv' ## save to path
path_pixc_isat2_erhai = 'data/eva_data/pixc_isat2_erhai.csv' ## save to path


In [39]:
isat2_wse_month_dianchi = read2month(path_isat2_wse_dianchi, var_name='isat2_wse')
isat2_wse_month_erhai = read2month(path_isat2_wse_erhai, var_name='isat2_wse')
isat2_wse_month_erhai


,isat2_wse
year_month,
2023-01,1965.728210
2023-02,1965.610718
2023-04,1965.238403
2023-05,1964.988037
2023-10,1966.324829
2023-11,1966.231140
2024-01,1966.052979
2024-04,1965.567993
2024-05,1965.399048


In [ ]:
## merge_pix_isat2
def merge_pixc_isat2(pixc_wse_month, isat2_wse_month):
    '''
    (1) merge pixc, isat2 data into one dataframe
    (2) system bias correction for pixc data by use isat2 data as reference
    '''
    ## merge lake heights from different sources
    pix_isat2_df = pixc_wse_month.merge(isat2_wse_month, on='year_month', how='left')
    # pix_isat2_df = pix_isat2_df.interpolate(method='linear', limit_area='inside')
    pix_isat2_df = pix_isat2_df.reindex(columns=['swot_pixc_wse', 'isat2_wse'])
    #计算均方根误差和相关性
    dif_pix_isat2 = pix_isat2_df['isat2_wse'] - pix_isat2_df['swot_pixc_wse']
    ## 计算均方根误差
    ## 计算差值的均值(系统偏差，以swot pixc为参考)
    mean_dif_pix_isat2 = dif_pix_isat2.mean()
    pix_isat2_df['isat2_wse_cor'] = pix_isat2_df['isat2_wse'] - mean_dif_pix_isat2
    ### write out
    return pix_isat2_df

pixc_isat2_dianchi = merge_pixc_isat2(pixc_wse_month_dianchi, isat2_wse_month_dianchi)
pixc_isat2_erhai = merge_pixc_isat2(pixc_wse_month_erhai, isat2_wse_month_erhai)
pixc_isat2_dianchi
# ## save to csv
# pixc_isat2_dianchi.to_csv(path_pixc_isat2_dianchi, index=True)
# pixc_isat2_erhai.to_csv(path_pixc_isat2_erhai, index=True)


### 3. 像素云面积加权平均 vs 普通平均   

In [109]:
## dianchi
path_unweighted_dianchi = 'data/swot_l2/pixc/dianchi-lake/dianchi_heights_simple_mean.pkl'  
path_dahiti_pix_lakesp_dianchi = 'data/eva_data/dahiti_pix_lakesp_dianchi.csv' ## 
path_dahiti_unweighted_weighted_dianchi = 'data/eva_data/dahiti_unweighted_weighted_dianchi.csv' ## save to path
## erhai
path_unweighted_erhai = 'data/swot_l2/pixc/erhai-lake/erhai_heights_simple_mean.pkl'
path_dahiti_pix_lakesp_erhai = 'data/eva_data/dahiti_pix_lakesp_erhai.csv' ## 
path_dahiti_unweighted_weighted_erhai = 'data/eva_data/dahiti_unweighted_weighted_erhai.csv' ## save to path


##### 2.1 所提方法获得结果（SWOT_pixc_wse, SWOT_pixc_wse_cor）

In [112]:
dahiti_pixc_lakesp_dianchi = pd.read_csv(path_dahiti_pix_lakesp_dianchi, index_col=0)
dahiti_pixc_lakesp_erhai = pd.read_csv(path_dahiti_pix_lakesp_erhai, index_col=0)
dahiti_pixc_lakesp_erhai


,dahiti_wse,swot_pixc_wse,swot_lakesp_wse,swot_pixc_wse_cor,swot_lakesp_wse_cor
year_month,,,,,
2023-09,1965.4342,1966.0171,NaN,1965.4158,NaN
2023-11,1965.5702,1966.3223,1966.4545,1965.7210,1965.723453
2023-12,1965.6812,1966.1672,1966.3710,1965.5659,1965.639953
2024-01,1965.5062,1966.0778,1966.4045,1965.4764,1965.673453
2024-02,1965.4037,1966.0978,1966.2340,1965.4965,1965.502953
2024-03,1965.3011,1965.8467,1965.9360,1965.2454,1965.204953
2024-04,1965.0922,1965.6117,1965.7800,1965.0104,1965.048953
2024-05,1964.8522,1965.4890,1965.5990,1964.8877,1964.867953
2024-06,1964.8743,1965.4530,1965.7430,1964.8517,1965.011953


##### 2.2 普通平均方法（非加权）所得结果

In [113]:
def read2df_unweighted(path_unweighted):
    '''
    (1) read unweighted data and convert to dataframe
    (2) convert decimal date to monthly date
    '''
    with open(path_unweighted, 'rb') as file:  
        unweighted_data = pickle.load(file)      
        df = pd.DataFrame.from_dict(unweighted_data, orient='index').reset_index()  
        df.columns = ['decimal_date', 'pixc_wse_unweighted']          
        # 转换日期
        df['date'] = df['decimal_date'].apply(decimal_to_date)  
        df['year_month'] = pd.to_datetime(df['date']).dt.to_period('M').astype(str)  # 转换为字符串格式 
        # 计算每月平均水位
        unweighted_wse_month = df.groupby('year_month')['pixc_wse_unweighted'].mean().reset_index()    
        unweighted_wse_month = unweighted_wse_month.set_index('year_month')
    return unweighted_wse_month


In [114]:
unweighted_wse_month_dianchi = read2df_unweighted(path_unweighted_dianchi)
unweighted_wse_month_erhai = read2df_unweighted(path_unweighted_erhai)
unweighted_wse_month_dianchi


,pixc_wse_unweighted
year_month,
2023-09,1887.776878
2023-10,1887.802006
2023-11,1887.716164
2023-12,1887.767118
2024-01,1887.696957
2024-02,1887.791682
2024-03,1887.701859
2024-04,1887.640440
2024-05,1887.500239


##### 2.3 数据合并及处理

In [115]:
def merge_dahiti_weighted_unweighted(dahiti_pixc_lakesp_df, pixc_unweighted_df):
    '''
    (1) merge dahiti_pixc_lakesp_df and pixc_unweight_df into one dataframe
    (2) calculate weighted and unweighted water level
    '''
    # 提取加权水位
    pixc_weighted_df = dahiti_pixc_lakesp_df[['dahiti_wse', 'swot_pixc_wse', 'swot_pixc_wse_cor']]
    pixc_weighted_df = pixc_weighted_df.rename(columns={'swot_pixc_wse': 'pixc_wse_weighted', 
                                                'swot_pixc_wse_cor': 'pixc_wse_weighted_cor'})
    # 合并加权和非加权数据
    pixc_weighted_unweighted_df = pixc_weighted_df.merge(pixc_unweighted_df, how='outer', on='year_month')
    ## interpolate missing values
    pixc_weighted_unweighted_df = pixc_weighted_unweighted_df.interpolate(method='linear', limit_area='inside')  
    # 非加权方法系统偏差改正（以dahiti为参考）
    dif_unweighted_dahiti = pixc_weighted_unweighted_df['pixc_wse_unweighted'] - pixc_weighted_unweighted_df['dahiti_wse']
    pixc_weighted_unweighted_df['pixc_wse_unweighted_cor'] = pixc_weighted_unweighted_df['pixc_wse_unweighted'] - dif_unweighted_dahiti.mean()
    return pixc_weighted_unweighted_df


In [116]:
dahiti_weighted_unweighted_dianchi = merge_dahiti_weighted_unweighted(dahiti_pixc_lakesp_dianchi, unweighted_wse_month_dianchi)
dahiti_weighted_unweighted_erhai = merge_dahiti_weighted_unweighted(dahiti_pixc_lakesp_erhai, unweighted_wse_month_erhai)
dahiti_weighted_unweighted_dianchi


,dahiti_wse,pixc_wse_weighted,pixc_wse_weighted_cor,pixc_wse_unweighted,pixc_wse_unweighted_cor
year_month,,,,,
2023-08,1887.4993,1887.8667,1887.5831,NaN,NaN
2023-09,1887.4312,1887.7599,1887.4763,1887.776878,1887.498950
2023-10,1887.5302,1887.7974,1887.5138,1887.802006,1887.524079
2023-11,1887.5972,1887.7504,1887.4668,1887.716164,1887.438236
2023-12,1887.4482,1887.7639,1887.4803,1887.767118,1887.489191
2024-01,1887.4261,1887.7036,1887.4200,1887.696957,1887.419029
2024-02,1887.4602,1887.7856,1887.5021,1887.791682,1887.513755
2024-03,1887.4592,1887.7152,1887.4316,1887.701859,1887.423931
2024-04,1887.3192,1887.6403,1887.3567,1887.640440,1887.362512


In [117]:
## accuracy     
## dianchi    
dif_weight_dahiti = dahiti_weighted_unweighted_dianchi['dahiti_wse']-dahiti_weighted_unweighted_dianchi['pixc_wse_weighted_cor']
dif_unweight_dahiti = dahiti_weighted_unweighted_dianchi['dahiti_wse']-dahiti_weighted_unweighted_dianchi['pixc_wse_unweighted_cor']
print('dianchi:', dif_weight_dahiti.std(), dif_unweight_dahiti.std())
## erhai    
dif_weight_dahiti = dahiti_weighted_unweighted_erhai['dahiti_wse']-dahiti_weighted_unweighted_erhai['pixc_wse_weighted_cor']
dif_unweight_dahiti = dahiti_weighted_unweighted_erhai['dahiti_wse']-dahiti_weighted_unweighted_erhai['pixc_wse_unweighted_cor']
print('erhai:', dif_weight_dahiti.std(), dif_unweight_dahiti.std())


dianchi: 0.05888572370179142 0.06406469601225408
erhai: 0.0863619829278778 0.09617982929139857


In [ ]:
# ## save to csv file. 
# dahiti_weighted_unweighted_dianchi.to_csv(path_dahiti_unweighted_weighted_dianchi, index=True)
# dahiti_weighted_unweighted_erhai.to_csv(path_dahiti_unweighted_weighted_erhai, index=True)

